In [1]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

'0.6.6.post1'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
token = "CENSORED"
model = 'meta-llama/Llama-3.1-8B-Instruct'
model = "BioMistral/BioMistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model)
llm = LLM(model=model, enforce_eager=True)

INFO 01-11 18:23:22 config.py:510] This model supports multiple tasks: {'embed', 'score', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 01-11 18:23:22 cuda.py:98] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 01-11 18:23:22 config.py:642] Async output processing is not supported on the current platform type cuda.
INFO 01-11 18:23:22 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='BioMistral/BioMistral-7B', speculative_config=None, tokenizer='BioMistral/BioMistral-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


/home/t-9morg0/miniconda3/envs/process-reports/lib/python3.10/site-packages/vllm/model_executor/model_loader/weight_utils.py:450: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

INFO 01-11 18:23:57 model_runner.py:1099] Loading model weights took 13.4966 GB
INFO 01-11 18:23:59 worker.py:241] Memory profiling takes 2.57 seconds
INFO 01-11 18:23:59 worker.py:241] the current vLLM instance can use total_gpu_memory (79.15GiB) x gpu_memory_utilization (0.90) = 71.24GiB
INFO 01-11 18:23:59 worker.py:241] model weights take 13.50GiB; non_torch_memory takes 0.10GiB; PyTorch activation peak memory takes 3.38GiB; the rest of the memory reserved for KV Cache is 54.26GiB.
INFO 01-11 18:23:59 gpu_executor.py:76] # GPU blocks: 27780, # CPU blocks: 2048
INFO 01-11 18:23:59 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 13.56x
INFO 01-11 18:24:01 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 4.49 seconds


In [12]:
# Read a reports csv and return it as a list of pairs
import csv
def reports_from_file(filename) -> list[tuple[str, str]]:
    data = []
    with open(filename, newline="") as reports:
        reader = csv.DictReader(reports)
        for row in reader:
            data.append((row["case_id"], row["text"]))
    return data
def reports_to_file(filename, data: list[tuple[str, str]]):
    with open(filename, "w+", newline="") as reports:
        writer = csv.DictWriter(reports, fieldnames=["case_id", "text"])
        writer.writeheader()
        for row in data:
            writer.writerow({"case_id": row[0], "text": row[1]})
reports = reports_from_file("TCGA_Reports_Processed.csv")
import random
random.seed(42)
random.shuffle(reports)
def write_for_compare(input_str: str, transformed: str):
    with open("raw_text.txt", "a+") as base_out:
        base_out.write(input_str + "\n")
        base_out.write("======END OF PROMPT======\n")
    with open("summarized.txt", "a+") as test_out:
        test_out.write(transformed + "\n")
        test_out.write("======END OF OUTPUT======\n")

def reset_compare():
    try:
        import os
        os.remove("raw_text.txt", )
        os.remove("summarized.txt")
    except OSError:
        return

PROMPT = "Please repeat the diagnosis, clinical history, and clinician comments from this report without rewording it or adding additional text:"
# PROMPT = "Please remove the 'gross description' from this report, if it is present:"
PROMPT = "Extract and repeat the results of the following clinical report in a single paragraph. Focus on test results, diagnoses and clinical history. Omit the gross description. Do not acknowledge this prompt."
ACK = "Here is the report summary:"


def make_prompt(report: str) -> str:
    return f"{PROMPT}\nINPUT:\n{report}\nINPUT_DONE\nOUTPUT:\n"
def make_example(example_report: str, fixed_report: str) -> str:
    return f"{make_prompt(example_report)}{fixed_report}\nOUTPUT_DONE\n===\n"
def make_prompt_for_chat(case: str, report: str) -> str:
    return f"\n{PROMPT}\n{case} - {report}\n"


In [4]:
NUM_TRAIN_EX = 10
reports_to_file("examples.csv", reports[-NUM_TRAIN_EX:])

In [13]:
import copy
examples = reports_from_file("examples.csv")
examples_fixed = reports_from_file("examples_fixed.csv")
assert(len(examples) == len(examples_fixed))
report_hints = [(case, e, ef) for (case, e), (_, ef) in zip(examples, examples_fixed)]

TRAIN_NUM = 3
random.shuffle(report_hints)
report_hints = report_hints[:TRAIN_NUM]


# Make the full prompt including the preprompt
# def make_full_prompt(report: str) -> str:
#     return preprompt + make_prompt(report)
# TYPOS = "fix typos"
# DIAGNOSTIC = "exclude sections like method notes and gross description"
# CONCISE = "make sentences more concise"
# preprompt = f"Summarize this pathological report, but first {', '.join([DIAGNOSTIC])}: \n\n"
# preprompt = "Extract diagnostic sections from this report without rewording it or adding additional text:\n"
chat_preprompt = [
    {
        "role": "system",
        "content": "Please clean up the following text medical reports. Please do not modify or add to the reports unless instructed otherwise."
    },
]
for case, ex_in, ex_out in report_hints:
    chat_preprompt.append({"role": "user", "content": make_prompt_for_chat(case, ex_in)})
    chat_preprompt.append({"role": "assistant", "content": f"{ACK}\n{case} - {ex_out}"})
# Mistral models don't do well with the chat history
chat_preprompt = []

testonly = True
NUM = 20 if testonly else len(reports)
convos = [copy.deepcopy(chat_preprompt) for _ in range(NUM)]
print(convos)
if testonly:
    # For tests, shuffle the reports so we can do a diversity of testing
    myreports = copy.deepcopy(reports)
    random.shuffle(myreports)
else:
    myreports = reports
for convo, (case, report) in zip(convos, myreports):
    convo.append({"role": "user", "content": make_prompt_for_chat(case, report)})
sampling_params = SamplingParams(max_tokens=1000, temperature=0, repetition_penalty=1.1)
outputs = llm.chat(messages=convos, sampling_params=sampling_params)
# outputs = []
# for convo in convos:
#     outputs.extend(llm.chat(conversations, sampling_params=sampling_params))

if testonly:
    reset_compare()
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        write_for_compare(prompt, generated_text)
        # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
else:
    new_reports = []
    for (case, _), output in outputs:
        generated_text = output.outputs[0].text
        generated_text = generated_text.replace("\n", " ")
        new_reports.append((case, generated_text))
    reports_to_file("TCGA_Reports_Transformed.csv", new_reports)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


Processed prompts: 100%|██████████| 20/20 [00:06<00:00,  2.92it/s, est. speed input: 3529.65 toks/s, output: 422.55 toks/s]


In [26]:
new_reports = []
for (case, _), output in zip(myreports, outputs):
    generated_text = output.outputs[0].text
    generated_text = generated_text.replace("\n", " ")
    new_reports.append((case, generated_text))
reports_to_file("TCGA_Reports_Transformed.csv", new_reports)

In [4]:
reports_transformed = reports_from_file("TCGA_Reports_Transformed.csv")